In [1]:
import numpy as np
from numpy import linalg as LA 

In [2]:
def mag(v):
    """ Calculate the magnitude of a vector

    Attributes: v: Numpy 1d array representing a vector
    Returns: the magnitude of the vector
    """
    return np.sqrt(v.dot(v))

def areaTriangle(p0, p1, p2):
    """ Calculate the area of a triangle

    Attributes: p0, p1, p2: Numpy arrays representing points of the triangle
    Returns: the total area of the triangle
    """
    return mag(np.cross(p2-p1, p0-p1))/2


In [3]:
p0 = np.array([1,2,3])
p1 = np.array([4,5,6])
p2 = np.array([7,8,-9])

In [4]:
areaTriangle(p0,p1,p2)

38.18376618407357

In [5]:
pixavg = np.zeros((8,3))

In [6]:
pixavg[0] = np.array((1,2,3))

In [7]:
np.mean(pixavg, axis=0)

array([0.125, 0.25 , 0.375])

In [8]:
class Halton:
    def next_prime():
        def is_prime(num):
            "Checks if num is a prime value"
            for i in range(2,int(num**0.5)+1):
                if(num % i)==0: return False
            return True

        prime = 3
        while(1):
            if is_prime(prime):
                yield prime
            prime += 2

    def vdc(n, base=2):
        vdc, denom = 0, 1
        while n:
            denom *= base
            n, remainder = divmod(n, base)
            vdc += remainder/float(denom)
        return vdc

    def halton_sequence(size, dim):
        seq = []
        primeGen = next_prime()
        next(primeGen)
        for d in range(dim):
            base = next(primeGen)
            seq.append([vdc(i, base) for i in range(size)])
        return seq


In [9]:
Halton.halton_sequence(8,2)

NameError: name 'next_prime' is not defined

In [10]:
maximum = np.array([4,5,6])
minimum = np.array([0,0,0])
p = np.array([2,3,4])
print (np.all(minimum <= p))
t = tuple(minimum)
print (t)

True
(0, 0, 0)


In [11]:
shape = np.array([10,10,10])
g = [[[[] for _ in range(shape[0])] for _ in range(shape[1])] for _ in range(shape[2])]
print (g[minimum[0]][minimum[1]][minimum[2]])

[]


In [12]:
o = np.array([-1,1,1])
d = np.array([1,0,0])
t0 = (minimum - o) / d
print(t0)
t1 = (maximum - o) / d
print(t1)
tmin = np.max(np.minimum(t0, t1))
print(tmin)
tmax = np.min(np.maximum(t0, t1))
print(tmax)
print((tmax-tmin) / shape)

[  1. -inf -inf]
[ 5. inf inf]
1.0
5.0
[0.4 0.4 0.4]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


In [13]:
size = maximum - minimum
if np.all(np.logical_and(minimum <= o, o <= maximum)):
    print("Inside box")
    index = np.floor((o - minimum) / size).astype(int)
else:
    print("Outside box")
    index = np.floor(((o + tmin * d) - minimum) / size).astype(int)
print(index)

Outside box
[0 0 0]


In [14]:
a = list(range(12))
l = a[: int(len(a) / 2)]
r = a[int(len(a) / 2) :]
print(l)
print(r)

[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]


In [20]:
    def samples(maximum, minimum, n):
        ret = []
        step = (maximum - minimum)/n
        step1 = np.array([step[0], 0.0, step[2]])
        step2 = np.array([0.0, step[1], 0.0])
        for i in range(n):
            for j in range(n):
                ret.append(minimum + ((0.5 + j) * step2) + ((0.5 + i) * step1))
        return ret

In [22]:
len(samples(np.array([5.0,5,5]),np.array([0.0,0.0,0.0]), 5))


25

In [6]:
def normalize(v):
    vLen = mag(v)
    if vLen == 0:
        return 0
    else:
        return v / vLen

In [39]:
class Rectangle:
    """ Simple geometric rectangle

    Attributes:
        kEpsilon: floating value used for allowable error in equality tests
        p: Origin point of the rectangle, where edges originate from
        a, b: Numpy arrays representing edges of the rectangle
        n: Numpy array representing unit normal vector
        mat: Tuple representing an RGB color with values in [0,255] (default: Blue)
    """
    kEpsilon = 0.0000001

    def __init__(self, p, a, b, mat=np.array([0, 0, 255])):
        """Initializes triangle attributes"""
        self.p = p
        self.a = a
        self.b = b
        self.aa = np.dot(a, a)
        self.bb = np.dot(b, b)
        self.n = normalize(np.cross(a, b))
        self.material = mat
        self.min = np.array([p, p + a, p + b, p + a + b]).min(axis=0)
        self.max = np.array([p, p + a, p + b, p + a + b]).max(axis=0)
        self.center = p + (a / 2) + (b / 2)

    def intersectRay(self, ray):
        """ Determine if a ray intersects the rectangle
            Returns: the parameter t for the intersection point to the ray
                     origin. Returns a value of None for no intersection
        """
        # Ray Tracing from the Ground Up, pg. 370
        t = np.dot(self.p - ray.o, self.n) / np.dot(ray.d, self.n)
        print (t)
        if t < self.kEpsilon:
            return None

        pt = ray.getPoint(t)
        print (pt)
        d = pt - self.p
        print (d)

        ddota = np.dot(d, self.a)
        print (ddota)
        if ddota < 0.0 or ddota > self.aa:
            return None

        ddotb = np.dot(d, self.b)
        print (ddotb)
        if ddotb < 0.0 or ddotb > self.bb:
            return None

        return t

    def getNormal(self, p):
        return self.n

In [40]:
class Ray:
    """ Simple ray class. Uses numpy arrays. Note that the direction may not be unit length"""

    def __init__(self, origin, direction):
        """ Initialize the ray with origin and direction"""
        self.o = origin
        self.d = direction

    def getPoint(self, t):
        """Returns the point along the ray generated by parameter t"""
        return self.o + t * self.d

In [41]:
r = Rectangle(np.array([-5.0, 0.0, 5.0]), np.array([0.0, 10.0, 0.0]), np.array([10.0, 0.0, 0.0]))

In [42]:
ray = Ray(np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 1.0]))

In [43]:
print(r.intersectRay(ray))

5.0
[0. 0. 5.]
[5. 0. 0.]
0.0
50.0
5.0
